# Extracting Correct (Patient's) keypoints

### Installs and Imports

In [1]:
!pip install ffmpeg-python
!pip install opencv-python

In [2]:
import os
import cv2
import json
import shutil
import subprocess
import numpy as np
from tqdm import tqdm
from google.colab import drive

### Mount Drive

In [3]:
drive.mount('/content/drive')
os.mkdir("/content/frames/")

Mounted at /content/drive


### Extract Frames

In [4]:
path = "/content/drive/MyDrive/ataxia_dataset"
# Extract frames for all videos & save them
for video in os.listdir(path):
  if ".mp4" not in video:
    continue
  else:
    # Break the video into frames at 30 fps
    if os.path.exists(f"/content/frames/{video[:-4]}/"):
      continue
    os.mkdir(f"/content/frames/{video[:-4]}/")
    command = ["ffmpeg", "-i", os.path.join(path, video), "-vf", "fps=30", f"/content/frames/{video[:-4]}/output_%05d.jpg"]
    out = subprocess.run(command, stderr=subprocess.PIPE)

### Get keypoints by unzipping them

In [6]:
!unzip "/content/drive/MyDrive/ataxia_dataset/keypoints.zip"

Streaming output truncated to the last 5000 lines.
  inflating: keypoints/24/output_00173_keypoints.json  
  inflating: keypoints/24/output_00078_keypoints.json  
  inflating: keypoints/24/output_00087_keypoints.json  
  inflating: keypoints/24/output_00092_keypoints.json  
  inflating: keypoints/24/output_00009_keypoints.json  
  inflating: keypoints/24/output_00164_keypoints.json  
  inflating: keypoints/24/output_00171_keypoints.json  
  inflating: keypoints/24/output_00072_keypoints.json  
  inflating: keypoints/24/output_00067_keypoints.json  
  inflating: keypoints/24/output_00100_keypoints.json  
  inflating: keypoints/24/output_00115_keypoints.json  
  inflating: keypoints/24/output_00098_keypoints.json  
  inflating: keypoints/24/output_00016_keypoints.json  
  inflating: keypoints/24/output_00003_keypoints.json  
  inflating: keypoints/24/output_00085_keypoints.json  
  inflating: keypoints/24/output_00090_keypoints.json  
  inflating: keypoints/24/output_00108_keypoints.json

### Lay keypoints on images for all frames, all videos

In [34]:
os.mkdir("/content/updated_frames/")

In [35]:
path = "/content/drive/MyDrive/ataxia_dataset"
for video in tqdm(os.listdir(path)):
  # Select the video to process
  if ".mp4" not in video:
    continue
  else:
    if os.path.exists(f"/content/updated_frames/{video[:-4]}/"):
      # if the video is already processed, then ignore
      continue
    os.mkdir(f"/content/updated_frames/{video[:-4]}/")
    # Otherwise make a directory to store the updated frames for that video
    # Updated frames = frames + keypoints on them.
    for _, frame in enumerate(os.listdir(f"/content/frames/{video[:-4]}/")): # Iterate over frames
      image = cv2.imread(f"/content/frames/{video[:-4]}/"+frame) # Open the frame image
      people = json.load(open("/content/keypoints/"+video[:-4]+"/"+frame[:-4]+"_keypoints.json", "r"))["people"] # Iterate over people
      for i, person in enumerate(people):
        person_kpts = np.array(person["pose_keypoints_2d"]).reshape(-1, 3)
        # Make it numpy array so we can process 3 points at once
        for (x, y, confidence) in person_kpts:
          cv2.circle(image, (int(x), int(y)), 3, (0, 255, 0), -1)
          cv2.putText(image, str(i), (int(x), int(y)), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 1)
      # Save the frame
      cv2.imwrite(f"/content/updated_frames/{video[:-4]}/output_{frame[8:-4]}.jpg", image)

100%|██████████| 155/155 [18:10<00:00,  7.04s/it]


### Make a video for each of them and store them

In [36]:
path = "/content/updated_frames/"
for frames in tqdm(os.listdir(path)):
  video_maker = ["ffmpeg", "-framerate", "1", "-pattern_type", "glob", "-i", path+f"{frames}/*.jpg", "-c:v", "libx264", "-pix_fmt", "yuv420p", path+f"{frames}/video.mp4"]
  subprocess.run(video_maker)

100%|██████████| 149/149 [37:27<00:00, 15.08s/it]


### Copy the first updated frame for each video to drive
Logic - Basically, if I have the first frame and I know ehich of the two people the patient is, in THAT frame, then I can pick from subsequent frames by the smaller norm, because the correct keypoints can't be the farther ones.

In [37]:
os.mkdir("/content/first_frames/")

In [40]:
for frames in tqdm(os.listdir(path)):
  copier = ["cp", f"/content/updated_frames/{frames}/output_0001.jpg", f"/content/first_frames/{frames}.jpg"]
  subprocess.run(copier)

100%|██████████| 149/149 [00:00<00:00, 276.17it/s]


In [45]:
!zip -r first_frames.zip first_frames/

  adding: first_frames/ (stored 0%)
  adding: first_frames/44.jpg (deflated 8%)
  adding: first_frames/55.jpg (deflated 6%)
  adding: first_frames/53.jpg (deflated 6%)
  adding: first_frames/115.jpg (deflated 16%)
  adding: first_frames/109.jpg (deflated 11%)
  adding: first_frames/138.jpg (deflated 15%)
  adding: first_frames/15.jpg (deflated 23%)
  adding: first_frames/104.jpg (deflated 18%)
  adding: first_frames/47.jpg (deflated 9%)
  adding: first_frames/20.jpg (deflated 14%)
  adding: first_frames/111.jpg (deflated 15%)
  adding: first_frames/30.jpg (deflated 16%)
  adding: first_frames/31.jpg (deflated 8%)
  adding: first_frames/86.jpg (deflated 19%)
  adding: first_frames/135.jpg (deflated 15%)
  adding: first_frames/22.jpg (deflated 12%)
  adding: first_frames/139.jpg (deflated 12%)
  adding: first_frames/3.jpg (deflated 16%)
  adding: first_frames/127.jpg (deflated 15%)
  adding: first_frames/40.jpg (deflated 8%)
  adding: first_frames/72.jpg (deflated 11%)
  adding: first_fr

In [46]:
shutil.copy(f"/content/first_frames.zip", "/content/drive/MyDrive/ataxia_dataset")

'/content/drive/MyDrive/ataxia_dataset/first_frames.zip'

### Thank you!